ÍNDICE:

1. Mapa de Google a partir de datos geocodificados
2. Visualización de redes e inteconexiones
3. Visualización de datos de correo

In [7]:
import urllib.request, urllib.parse, urllib.error
import http
import sqlite3
import json
import time
import ssl
import sys
import codecs

## 1. Mapa de Google a partir de datos geocodificados

In [8]:
api_key = False
# Si tiene una clave API de Google Places, ingrésela aquí
# api_key = 'AIzaSy___IDByT70'

if api_key is False:
    api_key = 42
    serviceurl = "http://py4e-data.dr-chuck.net/json?"
else :
    serviceurl = "https://maps.googleapis.com/maps/api/geocode/json?"

# Detalle adicional para urllib
# http.client.HTTPConnection.debuglevel = 1

conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('''
CREATE TABLE IF NOT EXISTS Locations (address TEXT, geodata TEXT)''')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

nombre_archivo = 'where.data'
ruta = fr'Datos\\Python para todos\\' + f'{nombre_archivo}'

manejador_archivos = open(ruta)
contador = 0

for linea in manejador_archivos:
    if contador > 200 :
        print('Recuperadas 200 ubicaciones, reinicia para recuperar más')
        break

    direccion = linea.strip()
    print('')
    cur.execute("SELECT geodata FROM Locations WHERE address= ?",
        (memoryview(direccion.encode()), ))

    try:
        datos = cur.fetchone()[0]
        print("Encontrado en la base de datos ",direccion)
        continue
    except:
        pass

    parametros = dict()
    parametros["direccion"] = direccion
    if api_key is not False: 
        parametros['llave'] = api_key
    url = serviceurl + urllib.parse.urlencode(parametros)

    print('Recuperando', url)
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    print('Recuperando', len(data), 'caracteres', data[:20].replace('\n', ' '))
    contador += 1

    try:
        js = json.loads(data)
    except:
        print(data)  # We print in case unicode causes an error
        continue

    if 'status' not in js or (js['status'] != 'OK' and js['status'] != 'ZERO_RESULTS') :
        print('==== Falla en recuperar ====')
        print(data)
        break

    cur.execute('''INSERT INTO Locations (address, geodata)
            VALUES ( ?, ? )''', (memoryview(address.encode()), memoryview(data.encode()) ) )
    conn.commit()
    if contador % 10 == 0 :
        print('Parando un rato...')
        time.sleep(5)

print("Ejecute geodump.py para leer los datos de la base de datos y poder visualizarlos en un mapa.")


Encontrado en la base de datos  Northeastern University

Encontrado en la base de datos  University of Hong Kong, Illinois Institute of Technology, Bradley University

Encontrado en la base de datos  Technion

Encontrado en la base de datos  Viswakarma Institute, Pune, India

Encontrado en la base de datos  UMD

Encontrado en la base de datos  Tufts University

Encontrado en la base de datos  Monash University

Encontrado en la base de datos  Kokshetau Institute of Economics and Management

Encontrado en la base de datos  RSU named S.A. Esenin

Encontrado en la base de datos  Tavrida National V.I. Vernadsky University

Encontrado en la base de datos  UOC

Encontrado en la base de datos  Irkutsk State University

Encontrado en la base de datos  Institute of Technology Telkom

Encontrado en la base de datos  Shanghai Jiao Tong University

Encontrado en la base de datos  University of Ilorin, Kwara State. Nigeria

Encontrado en la base de datos  Monash University Churchill Australia

Enc

Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Boston+University&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=The+University+of+Manchester&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Fachhochschule+D%C3%83%C2%BCsseldorf&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Pine+Manor+College+%28AA%29%2C+Harvard+University+%28BA%29%2C+Lesley+University+%28MEd%29&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Sim%C3%83%C2%B3n+Bol%C3%83%C2%ADvar+University&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Indiana+University+at+Bloomington&llave=42
Recuperando 13 cara

Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Universidade+Tecnica+de+Lisboa+-+Instituto+Superior+T%C3%83%C2%A9cnico&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Does+not+apply.&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=Stellenbosch+University&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=imt+ghazIABAD+INDIA&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=University+of+Pennsylvania&llave=42
Recuperando 13 caracteres No address...
No address...

Recuperando http://py4e-data.dr-chuck.net/json?direccion=National+Institute+of+Technology%2C+Jalandhar+%28India%29&llave=42
Recuperando 13 caracteres No address...
No address...

Rec

In [10]:
conn = sqlite3.connect('geodata.sqlite')
cur = conn.cursor()

cur.execute('SELECT * FROM Locations')

nombre_archivo = 'where.js'
ruta = fr'Datos\\Python para todos\\' + f'{nombre_archivo}'

manejador_archivos = codecs.open(ruta, 'w', "utf-8")
manejador_archivos.write("myData = [\n")

contador = 0
for fila in cur :
    data = str(fila[1].decode())
    try: 
        js = json.loads(str(data))
    except: 
        continue

    if not('status' in js and js['status'] == 'OK') : 
        continue

    latitud = js["results"][0]["geometry"]["location"]["lat"]
    longitud = js["results"][0]["geometry"]["location"]["lng"]
    if latitud == 0 or longitud == 0 : 
        continue
    where = js['results'][0]['formatted_address']
    where = where.replace("'", "")
    try :
        print(where, latitud, longitud)

        contador += 1
        if contador > 1 : manejador_archivos.write(",\n")
        output = "["+str(lat)+","+str(lng)+", '"+where+"']"
        manejador_archivos.write(output)
    except:
        continue

manejador_archivos.write("\n];\n")
cur.close()
manejador_archivos.close()
print(contador, "registros escritos en.js")
print("Abra where.html para ver los datos en un navegador.")

360 Huntington Ave, Boston, MA 02115, USA 42.339904 -71.0898892
Illinois, USA 40.6331249 -89.3985283
Haifa, 3200003, Israel 32.7767783 35.0231271
Vishwakarma Institute of Information Technology, Survey No. 2/3/4, VIM Private Rd, Kapil Nagar, Kondhwa, Pune, Maharashtra 411048, India 18.4595591 73.8854285
College Park, MD 20742, USA 38.9869183 -76.9425543
419 Boston Ave, Medford, MA 02155, USA 42.4085371 -71.1182729
Wellington Rd, Clayton VIC 3800, Australia -37.9142416 145.1346592
Kokshetau 020000, Kazakhstan 53.2871307 69.40444939999999
Rambla del Poblenou, 156, 08018 Barcelona, Spain 41.4065636 2.1945816
Ulitsa Karla Marksa, 1, Irkutsk, Irkutskaya oblast, Russia, 664003 52.2766643 104.2777445
China, Shang Hai Shi, Min Hang Qu, 老闵行东川路 邮政编码: 200240 31.0252201 121.4337784
P.M.B, 1515, Ilorin 240102, Kwara, Nigeria 8.4858957 4.674583
1 Northways Rd, Churchill VIC 3842, Australia -38.3090975 146.4255391
Urstein S 1, 5412 Salzburg, Austria 47.7233835 13.0871253
Korkeakoulunkatu 7 Kampusaree

## 2. Visualización de redes e inteconexiones 

## 3. Visualización de datos de correo 